# Hyper-parameter optimization with Vowpal Wabbit and Python

### Setup

1. Docker is awesome: 
    * https://docs.docker.com/mac/started/
2. Edit your virtual machine settings (may need to power off default vm)
    * System - add memory and processors
    * Network -> Port Forwarding
        * add entry - Name: jupyter, Protocol: TCP, Host IP: 127.0.0.1, Host Port: 8888, Guest Port: 8888
3. Start up a docker session
    * Docker Quick Start Terminal
3. Check out ML-Toolbox: 
    * git clone https://github.com/gramhagen/ml-toolbox
4. Start the notebook server:
    * docker-compose up -d
5. Open the Jupyter Notebook
    * open http://localhost:8888
    
### Cleanup
1. Stop the notebook server:
    * docker-compose stop
2. Power off the vm

### Get the data

In [21]:
from sklearn.datasets import load_svmlight_file
from sklearn.cross_validation import train_test_split
import pickle

with open('/notebook/data/n_features.pkl', 'r') as f:
    n_features = pickle.load(f)
    
feature_file = '/notebook/data/train-0.1m.svm'

X, y = load_svmlight_file(feature_file, n_features=n_features)
# convert to labels from 0.0, 1.0 to -1, 1
y = (y.astype(int) * 2 - 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.05, test_size=0.1, random_state=42)
X_tune, X_test, y_tune, y_test = train_test_split(X_test, y_test, train_size=0.5, test_size=0.5, random_state=42)


### Vanilla VW

In [22]:
from sklearn import metrics
from sklearn_vw import VWClassifier

model = VWClassifier()
model.fit(X_train, y_train)

print 'AUC: {}'.format(metrics.roc_auc_score(y_tune, model.predict(X_tune)))

AUC: 0.507892943671


### Parameter Search

#### Grid Search
http://scikit-learn.org/stable/auto_examples/model_selection/randomized_search.html

In [23]:
import numpy as np
from operator import itemgetter
from sklearn.grid_search import GridSearchCV
from time import time


# Utility function to report best scores
def report(grid_scores, n_top=3):
    top_scores = sorted(grid_scores, key=itemgetter(1), reverse=True)[:n_top]
    for i, score in enumerate(top_scores):
        print("Model with rank: {0}".format(i + 1))
        print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
              score.mean_validation_score,
              np.std(score.cv_validation_scores)))
        print("Parameters: {0}".format(score.parameters))
        print("")

        
# create parameter grid
params = {"l": [0.1, 1.0, 10],
          "power_t": [0, 0.5, 1.0]}

# run grid search
grid_search = GridSearchCV(VWClassifier(), 
                           param_grid=params, 
                           scoring='roc_auc')
start = time()
grid_search.fit(X_train, y_train)

print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(grid_search.grid_scores_)))
report(grid_search.grid_scores_)

GridSearchCV took 11.46 seconds for 9 candidate parameter settings.
Model with rank: 1
Mean validation score: 0.656 (std: 0.017)
Parameters: {'power_t': 0, 'l': 1.0}

Model with rank: 2
Mean validation score: 0.619 (std: 0.007)
Parameters: {'power_t': 0.5, 'l': 1.0}

Model with rank: 3
Mean validation score: 0.611 (std: 0.023)
Parameters: {'power_t': 0, 'l': 0.1}



In [43]:
model = VWClassifier(power_t=0, l=1.0)
model.fit(X_train, y_train)

print 'AUC: {}'.format(metrics.roc_auc_score(y_tune, model.predict(X_tune)))

AUC: 0.549069587455


#### Random Search

In [ ]:
from sklearn.grid_search import RandomizedSearchCV
from scipy.stats.distributions import uniform


# create parameter ranges
np.random.seed(1)
n_iter = 9
params = {"l": uniform(0.01, 10),
          "power_t": uniform()}

# run search
search = RandomizedSearchCV(VWClassifier(), 
                            param_distributions=params, 
                            scoring='roc_auc',
                            n_iter=n_iter)
start = time()
search.fit(X_train, y_train)

print("Parameter search took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(search.grid_scores_)))
report(search.grid_scores_)

In [44]:
model = VWClassifier(power_t=0.09, l=1.48)
model.fit(X_train, y_train)

print 'AUC: {}'.format(metrics.roc_auc_score(y_tune, model.predict(X_tune)))

AUC: 0.55413596137


#### Evolutionary Search
https://github.com/rsteca/sklearn-deap

In [ ]:
from evolutionary_search import EvolutionaryAlgorithmSearchCV
from scipy.stats.distributions import uniform

# create parameter grid
params = {"l": [0.1, 1.0, 10],
          "power_t": [0, 0.5, 1.0]}

search = EvolutionaryAlgorithmSearchCV(VWClassifier(), 
                                       param_grid=params, 
                                       scoring='roc_auc', 
                                       verbose=True, 
                                       n_jobs=1, 
                                       population_size=5)
start = time()
search.fit(X_train, y_train)

print("Parameter search took %.2f seconds for %d generations of settings."
      % (time() - start, search.generations_number))

#### Hyperopt-sklearn
https://github.com/hyperopt/hyperopt/wiki/FMin
or maybe this (http://hyperopt.github.io/hyperopt-sklearn/) ???
- Installation instructions: 
    - pip install hyperopt
    - pip install pymongo
    - pip install networkx

In [41]:
from hyperopt import hp, fmin, tpe, Trials

X_a, X_b, y_a, y_b = train_test_split(X_train, y_train, train_size=0.8, test_size=0.2, random_state=42)

param_names = ('power_t', 'l')
space = (hp.uniform('power_t', 0, 1),
         hp.uniform('l', 0.1, 10))

def obj_func(params):
    args = dict((k, params[i]) for i, k in enumerate(param_names))
    model = VWClassifier(**args)
    model.fit(X_a, y_a)
    return 1 - metrics.roc_auc_score(y_b, model.predict(X_b))

trials = Trials()
max_evals = 30

start = time()
best = fmin(obj_func, 
            space, 
            algo=tpe.suggest, 
            max_evals=max_evals, 
            trials=trials)

print("Parameter search took %.2f seconds for %d trials of settings."
      % (time() - start, max_evals))
print "Best metric: %f" % (1. - trials.best_trial['result']['loss'])
print "Best settings: %s" % trials.best_trial['misc']['vals']

Parameter search took 10.78 seconds for 30 trials of settings.
Best metric: 0.542935
Best settings: {'power_t': [0.02073725005871313], 'l': [8.993703550092983]}


In [42]:
model = VWClassifier(power_t=0.02, l=9)
model.fit(X_train, y_train)

print 'AUC: {}'.format(metrics.roc_auc_score(y_tune, model.predict(X_tune)))

AUC: 0.555773226664
